# **Big Data Final Project**

In [172]:
covid_df = spark.read.csv("gs://sourav988/host_train.csv",header=True)


In [173]:
covid_df.show

<bound method DataFrame.show of DataFrame[case_id: string, Hospital: string, Hospital_type: string, Hospital_city: string, Hospital_region: string, Available_Extra_Rooms_in_Hospital: string, Department: string, Ward_Type: string, Ward_Facility: string, Bed_Grade: string, patientid: string, City_Code_Patient: string, Type of Admission: string, Illness_Severity: string, Patient_Visitors: string, Age: string, Admission_Deposit: string, Stay_Days: string]>

The COVID-19 pandemic has placed an unprecedented strain on health systems, with rapidly increasing demand for healthcare in hospitals and intensive care units (ICUs) worldwide. As the pandemic escalates, determining the resulting needs for healthcare resources (beds, staff, equipment) has become a key priority for many countries. Projecting future demand requires estimates of how long patients with COVID-19 need different levels of hospital care.
While healthcare management has various use cases for using data science, patient length of stay is one critical parameter to observe and predict if one wants to improve the efficiency of the healthcare management in a hospital. 

The task is to accurately predict the Length of Stay for each patient on case by case basis so that the Hospitals can use this information for optimal resource allocation and better functioning. The length of stay is divided into 11 different classes ranging from 0-10 days to more than 100 days.

In [174]:
covid_df.count() # Total number of rows in our dataset

318438

In [175]:

covid_df.show(5) # The top 5 rows of the dataset

+-------+--------+-------------+-------------+---------------+---------------------------------+------------+---------+-------------+---------+---------+-----------------+-----------------+----------------+----------------+-----+-----------------+---------+
|case_id|Hospital|Hospital_type|Hospital_city|Hospital_region|Available_Extra_Rooms_in_Hospital|  Department|Ward_Type|Ward_Facility|Bed_Grade|patientid|City_Code_Patient|Type of Admission|Illness_Severity|Patient_Visitors|  Age|Admission_Deposit|Stay_Days|
+-------+--------+-------------+-------------+---------------+---------------------------------+------------+---------+-------------+---------+---------+-----------------+-----------------+----------------+----------------+-----+-----------------+---------+
|      1|       8|            2|            3|              2|                                3|radiotherapy|        R|            F|      2.0|    31397|              7.0|        Emergency|         Extreme|               2|51-

In [176]:
covid_df.dtypes  # The datatypes in our dataset

[('case_id', 'string'),
 ('Hospital', 'string'),
 ('Hospital_type', 'string'),
 ('Hospital_city', 'string'),
 ('Hospital_region', 'string'),
 ('Available_Extra_Rooms_in_Hospital', 'string'),
 ('Department', 'string'),
 ('Ward_Type', 'string'),
 ('Ward_Facility', 'string'),
 ('Bed_Grade', 'string'),
 ('patientid', 'string'),
 ('City_Code_Patient', 'string'),
 ('Type of Admission', 'string'),
 ('Illness_Severity', 'string'),
 ('Patient_Visitors', 'string'),
 ('Age', 'string'),
 ('Admission_Deposit', 'string'),
 ('Stay_Days', 'string')]

In [177]:
covid_df.describe().show()

+-------+---------------+------------------+------------------+------------------+------------------+---------------------------------+------------------+---------+-------------+------------------+-----------------+-----------------+-----------------+----------------+------------------+------+------------------+------------------+
|summary|        case_id|          Hospital|     Hospital_type|     Hospital_city|   Hospital_region|Available_Extra_Rooms_in_Hospital|        Department|Ward_Type|Ward_Facility|         Bed_Grade|        patientid|City_Code_Patient|Type of Admission|Illness_Severity|  Patient_Visitors|   Age| Admission_Deposit|         Stay_Days|
+-------+---------------+------------------+------------------+------------------+------------------+---------------------------------+------------------+---------+-------------+------------------+-----------------+-----------------+-----------------+----------------+------------------+------+------------------+------------------+
|

In [178]:
covid_df.summary().toPandas()  # Description of some statistics

,summary,case_id,Hospital,Hospital_type,Hospital_city,Hospital_region,Available_Extra_Rooms_in_Hospital,Department,Ward_Type,Ward_Facility,Bed_Grade,patientid,City_Code_Patient,Type of Admission,Illness_Severity,Patient_Visitors,Age,Admission_Deposit,Stay_Days
0,count,318438,318438,318438,318438,318438,318438,318438,318438,318438,318325,318438,313906,318438,318438,318438,318438,318438,318438
1,mean,159219.5,18.318840716246175,1.2568412061374585,4.771716943329627,0.7780980913081982,3.1976271676119055,None,None,None,2.625806958297338,65747.57947229916,7.251858836721821,None,None,3.2840992595104854,None,4880.749392346391,None
2,stddev,91925.276847557,8.633754885795021,1.535712038183553,3.1025353546703083,0.7525271792894237,1.1681714278597164,None,None,None,0.8731458045921742,37979.93644002417,4.745265561141447,None,None,1.7640613895591901,None,1086.7762536416217,None
3,min,1,1,0,1,0,0,TB & Chest disease,P,A,1.0,1,1.0,Emergency,Extreme,0,0-10,10010.0,0-10
4,25%,79606.0,11.0,0.0,2.0,0.0,2.0,None,None,None,2.0,32853.0,4.0,None,None,2.0,None,4186.0,None
5,50%,159203.0,19.0,1.0,5.0,1.0,3.0,None,None,None,3.0,65723.0,8.0,None,None,3.0,None,4741.0,None
6,75%,238836.0,26.0,2.0,7.0,1.0,4.0,None,None,None,3.0,98469.0,8.0,None,None,4.0,None,5409.0,None
7,max,99999,9,6,9,2,9,surgery,U,F,4.0,99999,9.0,Urgent,Moderate,9,91-100,9994.0,More than 100 Days


Converting some variable from string to int datatype

In [179]:
from pyspark.sql.types import IntegerType,BooleanType,DateType,FloatType

In [199]:
covid_df = covid_df.withColumn("Admission_Deposit",covid_df.Admission_Deposit.cast('int'))  

In [200]:
covid_df = covid_df.withColumn("Patient_Visitors",covid_df.Patient_Visitors.cast('int'))

In [201]:
covid_df = covid_df.withColumn("Available_Extra_Rooms_in_Hospital",covid_df.Available_Extra_Rooms_in_Hospital.cast('int'))

In [203]:
covid_df

DataFrame[case_id: string, Hospital: string, Hospital_type: string, Hospital_city: string, Hospital_region: string, Available_Extra_Rooms_in_Hospital: int, Department: string, Ward_Type: string, Ward_Facility: string, Bed_Grade: string, patientid: string, City_Code_Patient: string, Type of Admission: string, Illness_Severity: string, Patient_Visitors: int, Age: string, Admission_Deposit: int, Stay_Days: string]

In [204]:
covid_df_n = covid_df  # Making a new copy of the original dataset so as not to alter the original data

In [205]:
covid_df_n.count()

318438

Dropping the unimportant columns

In [206]:
covid_df_n = covid_df_n.drop("case_id")

In [207]:

covid_df_n = covid_df_n.drop("patientid")
covid_df_n = covid_df_n.drop("Hospital_region")


In [208]:
covid_df_n

DataFrame[Hospital: string, Hospital_type: string, Hospital_city: string, Available_Extra_Rooms_in_Hospital: int, Department: string, Ward_Type: string, Ward_Facility: string, Bed_Grade: string, City_Code_Patient: string, Type of Admission: string, Illness_Severity: string, Patient_Visitors: int, Age: string, Admission_Deposit: int, Stay_Days: string]

**Data Preprocessing**

In [128]:
# for categorical variables you can do one-hot encoding
from pyspark.ml.feature import OneHotEncoder, StringIndexer
# how to do scaling and prepare number columns for feeding a model
from pyspark.ml.feature import VectorAssembler, MinMaxScaler
# feature engineering
from pyspark.ml.pipeline import Pipeline
from pyspark.ml.feature import Tokenizer, HashingTF, IDF
from pyspark.ml.regression import LinearRegression

Pipeline for the categorical columns

In [210]:


pipeline = Pipeline(stages=[
    StringIndexer(inputCol="Department", outputCol="DepartmentIndex"),
    OneHotEncoder(inputCol="DepartmentIndex", outputCol="DepartmentOnehot"),
    
    StringIndexer(inputCol="Ward_Type", outputCol="Ward_TypeIndex"),
    OneHotEncoder(inputCol="Ward_TypeIndex", outputCol="Ward_TypeOnehot"),
    
    StringIndexer(inputCol="Hospital_city", outputCol="Hospital_cityIndex"),
    OneHotEncoder(inputCol="Hospital_cityIndex", outputCol="Hospital_cityOnehot"),
    
   StringIndexer(inputCol="City_Code_Patient", outputCol="City_Code_PatientIndex"),
    OneHotEncoder(inputCol="City_Code_PatientIndex", outputCol="City_Code_PatientOnehot"),
    
    StringIndexer(inputCol="Ward_Facility", outputCol="Ward_FacilityIndex"),
    OneHotEncoder(inputCol="Ward_FacilityIndex", outputCol="Ward_FacilityOnehot"),
    
    StringIndexer(inputCol="Hospital", outputCol="HospitalIndex"),
    
    StringIndexer(inputCol="Hospital_type", outputCol="Hospital_typeIndex"),
    
    
    StringIndexer(inputCol="Illness_Severity",outputCol="Illness_SeverityIndex"),  
    OneHotEncoder(inputCol="Illness_SeverityIndex", outputCol="Illness_SeverityOnehot"),
    
    StringIndexer(inputCol="Bed_Grade",outputCol="Bed_GradeIndex"),  
    OneHotEncoder(inputCol="Bed_GradeIndex", outputCol="Bed_GradeOnehot"),
    
    StringIndexer(inputCol="Age",outputCol="AgeIndex"),  
    OneHotEncoder(inputCol="AgeIndex", outputCol="AgeOnehot"),
    
    StringIndexer(inputCol="Type of Admission",outputCol="Type of AdmissionIndex"),  
    OneHotEncoder(inputCol="Type of AdmissionIndex", outputCol="Type of AdmissionOnehot"),
    
    VectorAssembler(
    inputCols=["City_Code_PatientOnehot","Hospital_cityOnehot","Type of AdmissionOnehot","AgeOnehot","DepartmentOnehot","Ward_TypeOnehot","Ward_FacilityOnehot","HospitalIndex","Hospital_typeIndex","Illness_SeverityOnehot","Bed_GradeOnehot"],
  
    outputCol="cat_features")
    
])

Converting the column Available Extra Rooms from float to int

In [213]:
covid_df_n = covid_df_n.withColumn("Available_Extra_Rooms_in_Hospital",covid_df_n.Available_Extra_Rooms_in_Hospital.cast('float'))

Pipeline for numerical values

In [215]:
from pyspark.ml.feature import VectorAssembler, MinMaxScaler
from pyspark.ml.feature import StandardScaler

pipeline1 = Pipeline(stages=[
    VectorAssembler(
        inputCols=["Available_Extra_Rooms_in_Hospital","Patient_Visitors","Admission_Deposit"],
        outputCol="pre_features"
    ),
    StandardScaler(
    inputCol="pre_features",outputCol="num_scaled"
    )
    
    
    
])

Pipeline for the target column

In [216]:
pipeline2 = Pipeline(stages=[
    
        
    
    StringIndexer(
        inputCol = 'Stay_Days', outputCol = 'labelIndex'
    )
    
    
    

])

Counting the number of null values in the dataset

In [219]:
df2 = covid_df_n.select([count(when(col(c).contains('None') | \
                            col(c).contains('NULL') | \
                            (col(c) == '' ) | \
                            col(c).isNull() | \
                            isnan(c), c 
                           )).alias(c)
                    for c in covid_df_n.columns])
df2.show()

+--------+-------------+-------------+---------------------------------+----------+---------+-------------+---------+-----------------+-----------------+----------------+----------------+---+-----------------+---------+
|Hospital|Hospital_type|Hospital_city|Available_Extra_Rooms_in_Hospital|Department|Ward_Type|Ward_Facility|Bed_Grade|City_Code_Patient|Type of Admission|Illness_Severity|Patient_Visitors|Age|Admission_Deposit|Stay_Days|
+--------+-------------+-------------+---------------------------------+----------+---------+-------------+---------+-----------------+-----------------+----------------+----------------+---+-----------------+---------+
|       0|            0|            0|                                0|         0|        0|            0|      113|             4532|                0|               0|               0|  0|                0|        0|
+--------+-------------+-------------+---------------------------------+----------+---------+-------------+---------+---

In [221]:
covid_df_n = covid_df_n.fillna(value="0")  # Filling the null values in the dataset

In [222]:
df2 = covid_df_n.select([count(when(col(c).contains('None') | \
                            col(c).contains('NULL') | \
                            (col(c) == '' ) | \
                            col(c).isNull() | \
                            isnan(c), c 
                           )).alias(c)
                    for c in covid_df_n.columns])
df2.show()

+--------+-------------+-------------+---------------------------------+----------+---------+-------------+---------+-----------------+-----------------+----------------+----------------+---+-----------------+---------+
|Hospital|Hospital_type|Hospital_city|Available_Extra_Rooms_in_Hospital|Department|Ward_Type|Ward_Facility|Bed_Grade|City_Code_Patient|Type of Admission|Illness_Severity|Patient_Visitors|Age|Admission_Deposit|Stay_Days|
+--------+-------------+-------------+---------------------------------+----------+---------+-------------+---------+-----------------+-----------------+----------------+----------------+---+-----------------+---------+
|       0|            0|            0|                                0|         0|        0|            0|        0|                0|                0|               0|               0|  0|                0|        0|
+--------+-------------+-------------+---------------------------------+----------+---------+-------------+---------+---

**Model Building**

In [271]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import RandomForestClassifier


final_pipeline = Pipeline(stages=[
    pipeline,pipeline1, pipeline2, 
    VectorAssembler(inputCols=["num_scaled", "cat_features"], outputCol="final_features"),
    
])

In [404]:
transformed_data = final_pipeline.fit(covid_df_n).transform(covid_df_n) # Transforming our dataset by passing it through the pipeline


In [354]:
sample_covid = transformed_data.limit(100000)  # Sampling 100000 rows 

Splitting the data to training and testing set

In [356]:
train, test = sample_covid.randomSplit([0.8, 0.2], seed = 2018)
print("Training Dataset Count: " + str(train.count()))
print("Test Dataset Count: " + str(test.count()))

Training Dataset Count: 80112


Test Dataset Count: 19888


**Random Forest Classifier**

In [348]:
rf = RandomForestClassifier(featuresCol = 'final_features', labelCol = 'labelIndex')
rfModel = rf.fit(train)
rfModel_train = rfModel.transform(train)
predictions = rfModel.transform(test)

In [349]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="labelIndex", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Accuracy = %g " % (accuracy))

Test Accuracy = 0.36473 


We get an accuracy of 36.4 % with Random forest Classifier

In [300]:
y_true2 = predictions.select(['labelIndex']).collect()
y_pred2 = predictions.select(['prediction']).collect()

In [301]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_true2, y_pred2))

              precision    recall  f1-score   support

         0.0       0.37      0.92      0.52      8728
         1.0       0.41      0.15      0.22      6861
         2.0       0.36      0.13      0.19      5297
         3.0       0.37      0.39      0.38      3485
         4.0       0.00      0.00      0.00      1813
         5.0       0.00      0.00      0.00      1091
         6.0       0.00      0.00      0.00      1040
         7.0       0.00      0.00      0.00       646
         8.0       0.00      0.00      0.00       429
         9.0       0.00      0.00      0.00       269
        10.0       0.00      0.00      0.00       276

    accuracy                           0.37     29935
   macro avg       0.14      0.14      0.12     29935
weighted avg       0.31      0.37      0.28     29935



/opt/conda/miniconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/miniconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/miniconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**Decision Tree classifier**

In [350]:

from pyspark.ml.classification import DecisionTreeClassifier

In [357]:
# Train a DecisionTree model.
dt = DecisionTreeClassifier(labelCol="labelIndex", featuresCol="final_features")

In [358]:
dtModel = dt.fit(train)
dtModel_train = dtModel.transform(train)
predictions = dtModel.transform(test)

In [359]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="labelIndex", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Accuracy = %g " % (accuracy))

Test Accuracy = 0.402152 


We get an accuracy of 40.2 % with Decision Trees

**OneVsRest classifier**

In [360]:
from pyspark.ml.classification import LogisticRegression, OneVsRest

In [361]:
# instantiate the base classifier.
lr = LogisticRegression(maxIter=10, tol=1E-6, fitIntercept=True)

# instantiate the One Vs Rest Classifier.
ovr = OneVsRest(classifier=lr,labelCol="labelIndex", featuresCol="final_features")
# train the multiclass model.
ovrModel = ovr.fit(train)

# score the model on test data.
predictions = ovrModel.transform(test)



In [362]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="labelIndex", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Accuracy = %g " % (accuracy))

Test Accuracy = 0.394157 


We get an accuracy of 40.2 % with OneVsRest Classifier

**Naive Bayes Classifier**

In [318]:
from pyspark.ml.classification import NaiveBayes

In [319]:
# create the trainer and set its parameters
nb = NaiveBayes(smoothing=1.0, modelType="multinomial",labelCol="labelIndex", featuresCol="final_features")

In [320]:
nbModel = nb.fit(train)

predictions = nbModel.transform(test)

In [321]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="labelIndex", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Accuracy = %g " % (accuracy))

Test Accuracy = 0.332153 


We get an accuracy of 40.2 % with Naive Bayes Classifier

The best classifier is the Decision Tree. So we will tune the hyperparameters for this model

In [398]:
# my param grid, 
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.tuning import TrainValidationSplit

param_grid = (
    ParamGridBuilder()
    .addGrid(dt.maxDepth, [3,5,10,20])
    
    .build()
)

In [399]:
tvs = TrainValidationSplit(estimator=dt,
                           estimatorParamMaps=param_grid,
                           evaluator=MulticlassClassificationEvaluator(labelCol="labelIndex",metricName="accuracy"),
                           # 80% of the data will be used for training, 20% for validation.
                           trainRatio=0.7,
                          collectSubModels=True)

# Running TrainValidationSplit, and choosing the best set of parameters.
model = tvs.fit(train)

22/01/17 22:32:30 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 1354.7 KiB
22/01/17 22:32:32 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 1832.6 KiB
22/01/17 22:32:33 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 2.4 MiB
22/01/17 22:32:35 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 3.0 MiB
22/01/17 22:32:36 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 3.8 MiB
22/01/17 22:32:38 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 4.6 MiB
22/01/17 22:32:40 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 5.4 MiB
22/01/17 22:32:42 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task binary with size 6.3 MiB
22/01/17 22:32:44 WARN org.apache.spark.scheduler.DAGScheduler: Broadcasting large task bi

In [400]:
predictions = model.transform(test) # Using our best model 

In [401]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="labelIndex", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Accuracy = %g " % (accuracy))

Test Accuracy = 0.416382 


In [402]:
y_true2 = predictions.select(['labelIndex']).collect()
y_pred2 = predictions.select(['prediction']).collect()

In [403]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_true2, y_pred2))

              precision    recall  f1-score   support

         0.0       0.43      0.74      0.54      5849
         1.0       0.41      0.38      0.40      4533
         2.0       0.38      0.22      0.28      3466
         3.0       0.42      0.47      0.44      2316
         4.0       0.31      0.12      0.18      1227
         5.0       0.07      0.00      0.01       733
         6.0       0.31      0.03      0.05       697
         7.0       0.54      0.35      0.43       420
         8.0       0.33      0.24      0.28       297
         9.0       0.20      0.01      0.02       172
        10.0       0.12      0.01      0.01       178

    accuracy                           0.42     19888
   macro avg       0.32      0.23      0.24     19888
weighted avg       0.39      0.42      0.38     19888



The maximum accuracy we can get from tuning the hyperparameters is 42%.